In [1]:
print('hi')

hi


In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
google_api_key = os.getenv('GOOGLE_API_KEY')

In [4]:
if google_api_key == "":
    print('Api key not available')
else:
    print('api key found')

api key found
